<a href="https://colab.research.google.com/github/marioschuele/masterarbeit/blob/main/SOTA_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd
import datetime

%load_ext tensorboard

In [ ]:
# Install package Kaggle for dataset download
! pip install kaggle -q

#Download and unzip Kaggle dataset

! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download yuweisunut/sidd-segmented-intrusion-detection-dataset
!unzip sidd-segmented-intrusion-detection-dataset

In [ ]:
directory = 'SIDD'

imgs = {}
uid = 0

for client in os.listdir(directory):
  curr_path = f'{directory}/{client}/pcap'

  for subdir in os.listdir(curr_path):
    curr_path = f'{directory}/{client}/pcap/{subdir}/dataset'
    curr_type = subdir[-1:]
    if curr_type == str(1):
        
        for dayscen in os.listdir(curr_path):
          curr_path = f'{directory}/{client}/pcap/{subdir}/dataset/{dayscen}'

          for img in os.listdir(curr_path):
            if dayscen == 'benign':
                imgs[uid] = {'id': uid, 'label': str(0), 'fn': img, 'path': curr_path + '/' + img}
            elif dayscen == 'malicious':
                imgs[uid] = {'id': uid, 'label': str(curr_type), 'fn': img, 'path': curr_path + '/' + img}
            uid +=1


In [ ]:
img_df = pd.DataFrame.from_dict(imgs,orient='index')
img_df['label'] = img_df['label'].astype(int)
#img_df.loc[img_df.index[(img_df['label']==3)],'label'] = 2
print(len(img_df))

In [ ]:
def _parse_function(filename, label):
    image_string = tf.io.read_file(filename)
    image_decoded = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.cast(image_decoded, tf.float32)
    return image, label

file_paths = img_df.path
file_labels = img_df["label"]

dataset = tf.data.Dataset.from_tensor_slices((file_paths, file_labels))

dataset = dataset.map(_parse_function)
dataset = dataset.batch(32)
dataset.element_spec

In [ ]:
#Distribute dataset into train, validation and test set
def get_dataset_partitions_tf(ds, ds_size, train_split, val_split, test_split, shuffle, shuffle_size):
    assert (train_split + test_split + val_split) == 1
    
    if shuffle:
        # Specify seed to always have the same split distribution between runs
        ds = ds.shuffle(shuffle_size, seed=12)
        prefetch_size = tf.data.AUTOTUNE
        ds = ds.prefetch(prefetch_size)
    
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)    
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    

    print('Size of {}:  {}'.format('train ds', str(train_ds.cardinality().numpy())))
    print('Size of {}:  {}'.format('val ds', str(val_ds.cardinality().numpy())))
    print('Size of {}:  {}'.format('test ds', str(test_ds.cardinality().numpy())))
    
    return train_ds, val_ds, test_ds

train_ds, val_ds, test_ds = get_dataset_partitions_tf(ds=dataset, ds_size=dataset.cardinality().numpy(), train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000)

In [ ]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# Create a TensorBoard callback
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


model = tf.keras.applications.EfficientNetB3(
    include_top=True,
    weights=None,
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=2,
    classifier_activation="sigmoid"
)


model.compile(
    optimizer=keras.optimizers.SGD(),  #learning_rate = 0.01
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']        #keras.metrics.sparse_categorical_accuracy
)


history = model.fit(
    x=train_ds,
    epochs=20,
    validation_data=val_ds,
    callbacks=[tensorboard_callback]
)

evaluation = model.evaluate(test_ds)

In [ ]:
%tensorboard --logdir logs/fit/

In [ ]:
#Plot accuracy of model
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

#Plot loss of model
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()